# Determining the Effectiveness of the High Speed Test Track testing for the determination of accelerometer error coefficients. 
### By Sean Abrahamson 

This is a jupyter note book walking through the code for simulating an the performace of a single accelerometer output going down the Holloman High Speed Test Track and then using a the 746 TS Reference Position Vector to computer the error coefficients using least squares. 



### Import necessary libaries and functions from custom functions and classes from other jupiter notebooks.

In [1]:
import os.path
import pandas as pd
import numpy as np 
import plotly.graph_objects as go
import pickle
import pdb

# Import other Jupyter Notebooks
%run Thesis_Main.ipynb

### Set initial coefficients and parameters for script

Set the initial configuration parameters and logic that drives how results are computed. 

In [2]:
############################################################
#%% Initial Configuration Parameters
############################################################

# Set value for g

g = 9.791807  


# Set parameters for error added to Reference Position Vector

sigmaRPV = 0.01        # Standard deviation of Random noise centered at zero added to downtrack distance (meters)
tauRPV =  0            # Time Lag Error (seconds)
biasRPV = 0            # Bias error in RPV (meters) 

# Set number of Monte Carlo Runs

MCnum = 10

# Set custom coefficients for Accelerometer error model. Updates accelerometer coefficient error model 
# dictionary if ChangeDefaultCoeff is set to True.

CoeffDict = {'K_0': 1}

# Used to determine how many coefficients to calculate.

N_model_start = 2     #  0 =  K_1 (Scale Factor), 1 = K_0 (Bias), 2 = K_2, etc. 
N_model_end = 5      #  0 = K_1 (Scale Factor), 1 = K_0 (Bias), 2 = K_2, etc.


# Clean up Model indicies and define Error Coefficient Names
N_model = [0,0]
# Fix indexing numbers
N_model[0] = N_model_start  ### REVIEW THIS
N_model[1]= N_model_end + 1

# Definition of corresponding coefficient names that will be computed based on above pararmeters
ModelList = ['K_1', 'K_0', 'K_2', 'K_3','K_4','K_5']


############################################################
#%% Initial Configuration Logic
############################################################

# If set to True, accelerometer model error will be updated with CoeffDict values set in intial parameters.

changeDefaultCoeff = False


# Generate New Trajectory. If set to True new Trajectory will be created and saved to .pkl file from EGI data.

generateNewTrajectory = False


# Generate New RPV. If set to True a new RPV will be generated and saved to .pkl file. If set to False, code will 
# to make sure and RPV with the parameters set in the intial configuration is available. If not availble a new RPV 
# be generated. 
generateNewRPV = True


# LeastSquaresMethod sets the method used for Least Squares Regression analaysis. Default is set to 'LongHand'
#  - 'LongHand':  Computes the least squares using numpy matrix multiplication. This is the only method that works for 
#                 Weighted Least Squares with correleated off diagonal values in the weighting matrix.
#  - 'Numpy':     Uses the least squares function from the numpy.linalg library. This method should not be used if using any sort of weighted least squares method.
#  - 'SciKit':    Computes the least squares regression using the SciKit library. This does not use any correlated off diagonal values. 

LeastSquaresMethod = 'LongHand'

# If set to True the least squares regression method for determining error coefficients will use a 
# Weighted Least Squares Method.

WLS = True

# If set to true the model will perform regression analysis for each term indiviually as well as the full model as defined above or look at each individual coefficient.

individualCoeffAnalysis = True

## Run Monte Carlo Simulations


In [ ]:
# Intialize Data Storage variables
coefficientEstimates = [];

# Run Monte Carlo Sims
for mc in range(MCnum):

    print(f'Running MC {mc}...', end="\r")  
    
    ########################t######################################################################################
    #%% Generate or import trajectory
    if generateNewTrajectory == True:      
        generateReferenceTrajectory()

    # Import Reference Trajectory
    referenceTrajectory = pd.read_pickle("./referenceTrajectory.pkl")


    ##############################################################################################################
    #%% Generate track reference position vectory

    # If generateNewRPV is set to False, check if an RPV exists with the specified parameters. If it does not
    # then set generateNewRPV to True so tha generateNewRPV runs anyways.
    if generateNewRPV == False:   
        generateNewRPV = not os.path.isfile(f"./RPVs/trackRPV_sig{sigmaRPV}_tau{tauRPV}_bias{biasRPV}.pkl")

    if generateNewRPV == True:    
        generateTrackRPV(referenceTrajectory, sigmaRPV, tauRPV, biasRPV)

    # Import trackRPV pickle file that matches configuration parameters
    trackRPV = pd.read_pickle(f"./RPVs/trackRPV_sig{sigmaRPV}_tau{tauRPV}_bias{biasRPV}.pkl")


    ##############################################################################################################
    #%% Generate Simulated Accelerometer for full model
    sensorSim, AccelObj = AccelSim(referenceTrajectory, N_model, changeDefaultCoeff, CoeffDict, g)
    
    
    #%% Perform Regression Analysis for full model
    if mc == 0:
        coefficientDF, Error, cov_A, A, Ve_x, W, LeastSquaresMethod = RegressionAnalysis(referenceTrajectory, trackRPV, AccelObj, sensorSim, N_model, g, sigmaRPV, WLSoption = WLS, computeCovariance = True)
        coefficientCovariance = cov_A
    else:
        coefficientDF, Error, cov_A, A, Ve_x, W, LeastSquaresMethod = RegressionAnalysis(referenceTrajectory, trackRPV, AccelObj, sensorSim, N_model, g, sigmaRPV, WLSoption = WLS, computeCovariance = False)
  
    coefficientEstimates.append(coefficientDF)
    
print('Completed MC runs')
    

> /var/folders/q1/b08x7v8d7vdfddvg49_v4hq00000gn/T/ipykernel_36020/2782600862.py(135)RegressionAnalysis()
    133     pdb.set_trace()
    134 
--> 135     print_List = np.array(list(coeff_dict.keys()))
    136 
    137     n = 0

ipdb> w
    [... skipping 21 hidden frame(s)]

  /var/folders/q1/b08x7v8d7vdfddvg49_v4hq00000gn/T/ipykernel_36020/3381718508.py(40)<module>()
     38     #%% Perform Regression Analysis for full model
     39     if mc == 0:
---> 40         coefficientDF, Error, cov_A, A, Ve_x, W, LeastSquaresMethod = RegressionAnalysis(referenceTrajectory, trackRPV, AccelObj, sensorSim, N_model, g, sigmaRPV, WLSoption = WLS, computeCovariance = True)
     41         coefficientCovariance = cov_A
     42     else:

> /var/folders/q1/b08x7v8d7vdfddvg49_v4hq00000gn/T/ipykernel_36020/2782600862.py(135)RegressionAnalysis()
    133     pdb.set_trace()
    134 
--> 135     print_List = np.array(list(coeff_dict.keys()))
    136 
    137     n = 0

ipdb> print(w)
[[ 0.00086744 -0.0048

# Monte Carlo Results Analysis

In [ ]:
CoefficientEstimateErrors = pd.DataFrame()

K_1_Errors = []
K_0_Errors = []
K_2_Errors = []
K_3_Errors = []
K_4_Errors = []
K_5_Errors = []

for n in range(len(coefficientEstimates)):
    K_1_Errors.append(coefficientEstimates[n]['Coefficient Estimate Error']['K_1'])
    K_0_Errors.append(coefficientEstimates[n]['Coefficient Estimate Error']['K_0'])
    K_2_Errors.append(coefficientEstimates[n]['Coefficient Estimate Error']['K_2'])
    K_3_Errors.append(coefficientEstimates[n]['Coefficient Estimate Error']['K_3'])
    K_4_Errors.append(coefficientEstimates[n]['Coefficient Estimate Error']['K_4'])
    K_5_Errors.append(coefficientEstimates[n]['Coefficient Estimate Error']['K_5'])
    
CoefficientEstimateErrors['K_1_Errors'] = K_1_Errors
CoefficientEstimateErrors['K_0_Errors'] = K_0_Errors
CoefficientEstimateErrors['K_2_Errors'] = K_2_Errors
CoefficientEstimateErrors['K_3_Errors'] = K_3_Errors
CoefficientEstimateErrors['K_4_Errors'] = K_4_Errors
CoefficientEstimateErrors['K_5_Errors'] = K_5_Errors

In [ ]:
print(CoefficientEstimateErrors['K_5_Errors'].std()**2)

In [ ]:
print(np.diag(coefficientCovariance))

In [ ]:
print(coefficientEstimates[0])